In [1]:
import os, sys, shutil
sys.path.append(os.path.join(os.getcwd(),"inputs"))
sys.path.append(os.path.join(os.getcwd(),r"../../../scripts/summarize/standard"))
sys.path.append(os.path.join(os.getcwd(),r'../../..'))
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import sqlalchemy
from shapely import wkt
from input_configuration import *
from emme_configuration import *
from standard_summary_configuration import *
pd.options.mode.chained_assignment = None
import os, sys, time, shutil
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
pd.options.mode.chained_assignment = None
import h5py

%matplotlib inline

from IPython.display import display, HTML

if (sys.version_info < (3, 0)):
    HTML('''<script>
    code_show=true; 
    function code_toggle() {
     if (code_show){
     $('div.input').hide();
     } else {
     $('div.input').show();
     }
     code_show = !code_show
    } 
    $( document ).ready(code_toggle);
    </script>
    <form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')


In [2]:
CSS = """
.output {
    flex-direction: row;
}
"""

HTML('<style>{}</style>'.format(CSS))

# Relative path between notebooks and goruped output directories
relative_path = '../../../outputs'
survey_path = '../../../inputs/base_year/survey'

In [3]:
def load_network_summary(filepath):
    """Load network-level results using a standard procedure. """
    df = pd.read_csv(filepath)

    # Congested network components by time of day
    df.columns

    # Get freeflow from 20to5 period

    # Exclude trips taken on non-designated facilities (facility_type == 0)
    # These are artificial (weave lanes to connect HOV) or for non-auto uses 
    df = df[df['data3'] != 0]    # data3 represents facility_type

    # calculate total link VMT and VHT
    df['VMT'] = df['@tveh']*df['length']
    df['VHT'] = df['@tveh']*df['auto_time']/60

    # Define facility type
    df.loc[df['data3'].isin([1,2]), 'facility_type'] = 'highway'
    df.loc[df['data3'].isin([3,4,6]), 'facility_type'] = 'arterial'
    df.loc[df['data3'].isin([5]), 'facility_type'] = 'connector'

    # Calculate delay
    # Select links from overnight time of day
    delay_df = df.loc[df['tod'] == '20to5'][['ij','auto_time']]
    delay_df.rename(columns={'auto_time':'freeflow_time'}, inplace=True)

    # Merge delay field back onto network link df
    df = pd.merge(df, delay_df, on='ij', how='left')

    # Calcualte hourly delay
    df['total_delay'] = ((df['auto_time']-df['freeflow_time'])*df['@tveh'])/60    # sum of (volume)*(travtime diff from freeflow)

    df['county'] =df['@countyid'].map({33: 'King',
                                      35: 'Kitsap',
                                      53: 'Pierce',
                                      61: 'Snohomish'})
    
    return df

In [4]:
pd.options.display.float_format = '{:0,.0f}'.format

# Regional Emissions

In [5]:
df = pd.read_csv(os.path.join(relative_path,'emissions',r'emissions_summary.csv'))
df = df.groupby('pollutant_name').sum()
df.rename(columns={'start_tons': 'Start', 'intrazonal_tons': 'Intrazonal', 'interzonal_tons': 'Interzonal',
                  'total_daily_tons': 'Total Daily'}, inplace=True)
df = df[df.index.isin(['CO','NOx','PM25 Total','PM10 Total','CO2 Equivalent','VOCs'])]
df = df.applymap(lambda x: x if x > 100 else str(round(x,1)))
df

,Start,Intrazonal,Interzonal,Total Daily
pollutant_name,,,,
CO,116,1.1,153,270
CO2 Equivalent,"2,615",198,"34,160","36,973"
NOx,7.5,0.1,22.3,29.8
PM10 Total,0.5,0.0,5.1,5.6
PM25 Total,0.5,0.0,1.0,1.4
VOCs,7.8,0.0,2.2,10.0


# Emissions by Vehicle Type

In [ ]:
# Calculate emissions and VMT by vehicle type and save results
# Note that Total VMT will not match regional totals because we are not included buses in the emissions summaries

df = pd.read_csv(os.path.join(relative_path,'emissions',r'emissions_summary.csv'))
df = df.groupby(['pollutant_name','veh_type']).sum()
df.rename(columns={'start_tons': 'Start', 'intrazonal_tons': 'Intrazonal', 'interzonal_tons': 'Interzonal',
                  'total_daily_tons': 'Total Daily'}, inplace=True)
df = df.reset_index()
df = df[df.pollutant_name.isin(['CO','NOx','PM25 Total','PM10 Total','CO2 Equivalent','VOCs'])]
# df
df_network = load_network_summary(os.path.join(relative_path,'network',r'network_results.csv'))

df_network['light_vmt'] = df_network[['@hov2_inc1','@hov2_inc2', '@hov2_inc3', '@hov3_inc1', '@hov3_inc2', '@hov3_inc3',
          '@sov_inc1', '@sov_inc2', '@sov_inc3', '@tnc_inc1', '@tnc_inc2','@tnc_inc3']].sum(axis=1)
df_network['light_vmt'] = df_network['light_vmt']*df_network['length']

df_network['medium_vmt'] = df_network['@mveh']*df_network['length']

df_network['heavy_vmt'] = df_network['@hveh']*df_network['length']

_df = pd.DataFrame([df_network['light_vmt'].sum(),
              df_network['medium_vmt'].sum(),
              df_network['heavy_vmt'].sum()])
_df.index = ['light','medium','heavy']
_df.columns=['VMT']
_df['veh_type'] = _df.index

df = df.merge(_df, on='veh_type', how='left')

df.to_csv(os.path.join(relative_path,'emissions',r'emissions_by_vehtype_summary.csv'))

df.rename(columns={'pollutant_name': 'Pollutant', 'veh_type': 'Vehicle Type',
                   'Total Daily': 'Total Daily (Tons)'}, inplace=True)
col_list = ['Start','Intrazonal','Start','Intrazonal','Interzonal','Total Daily (Tons)', 'VMT']
df[col_list] = df[col_list].applymap(lambda x: x if x > 100 else str(round(x,1)))
df